## From previouse notebook

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

RANDOM_STATE = 42
MIN_SAMPLE_SPLIT=4
MIN_SAMPLES_LEAF=5
N_ESTIMATORS=100
N_SPLITS = 5
PATH = '../data/train.csv'
df_raw = pd.read_csv(PATH)


def impute_age(df, value):
    '''
    Replaces Nulls in column "Age" of a dataframe with the passed value

    Parameters:
        dataframe (pandas.DataFrame): DataFrame on which to operate
        value (float): Value used for imputation
    Returns:
        pandas.DataFrame
    '''

    df['Age'] = df["Age"].fillna(value)
    return df


def convert_sex(df):
    '''
    Replacing sex in column "Sex" of a dataframe to 1 if it's male and 0 if it's female

    Parameters:
        dataframe (pandas.DataFrame): Dataframe on which to operate
    Returns:
        pandas.DataFrame
    '''

    df['is_male'] = 0
    df.loc[df['Sex'] == 'male', 'is_male'] = 1
    df = df.drop(columns=['Sex'])
    return df

def transform_data(df, mean_age_value):
    '''
    Applying data cleaning functions to data sets

    Paramters:
        dataframe (pandas.DataFrame): Dataframe on which to operate
        mean_age (float): Mean age of training data set
    Retruns:
        pandas.DataFrame
    '''

    df = impute_age(df, mean_age_value)
    df = convert_sex(df)
    return df


features = ['Age', 'Sex', 'Pclass']
LABEL = 'Survived'

X = df_raw[features]
y = df_raw[LABEL]

k_fold = KFold(
    n_splits=N_SPLITS,
    shuffle=True,
    random_state=RANDOM_STATE
)

scores = []

for train_index, test_index in k_fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    mean_age = X_train['Age'].mean()

    X_train = impute_age(X_train, mean_age)
    X_train = convert_sex(X_train)

    X_test = impute_age(X_test, mean_age)
    X_test = convert_sex(X_test)

    clf = RandomForestClassifier(n_estimators=N_ESTIMATORS, bootstrap=True, criterion='entropy',
                                min_samples_leaf=MIN_SAMPLES_LEAF,
                                min_samples_split=MIN_SAMPLE_SPLIT, random_state=RANDOM_STATE)

    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)

    acc_score = round(accuracy_score(y_test, y_predict),3)

    print(acc_score)

    scores.append(acc_score)

print()
print("Average:", round(100*np.mean(scores), 1), "%")
print("Std:", round(100*np.std(scores), 1), "%")

0.844
0.781
0.854
0.77
0.803

Average: 81.0 %
Std: 3.3 %


## Count relatives based on sibsp and parch

In [2]:
def count_relatives_on_board(df):
    df["RelativesOnboard"] = df["SibSp"] + df["Parch"]
    return df

In [3]:
df = count_relatives_on_board(df_raw)

In [4]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,RelativesOnboard
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,3
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0


## Change titles to Miss and Mr cryptonims

In [5]:
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
      'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}

In [6]:
def set_title(df):
    df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=True)
    df.replace({'Title': mapping}, inplace=True)
    return df

In [7]:
df = set_title(df_raw)

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
def title_encode(df):
    label_encoder = LabelEncoder()
    df['Title'] = label_encoder.fit_transform(df['Title'])
    return df

In [10]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,RelativesOnboard,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,Rev
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,3,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0,Mr


# scaling values

In [11]:
from sklearn.preprocessing import StandardScaler
def scaling_values(df):
    scaler = StandardScaler()
    scaler.fit(df)
    scaled_data = scaler.fit_transform(df)
    scaled_data = pd.DataFrame(scaled_data, columns=df.columns)
    scaled_data['Survived'] = df['Survived']
    return scaled_data

## Apply Changes

In [12]:
def transform_data(df, mean_age_value):
    '''
    Applying data cleaning functions to data sets

    Paramters:
        dataframe (pandas.DataFrame): Dataframe on which to operate
        mean_age (float): Mean age of training data set
    Retruns:
        pandas.DataFrame
    '''
    df = set_title(df)
    df = count_relatives_on_board(df)
    df = impute_age(df, mean_age_value)
    df = convert_sex(df)
    df = title_encode(df)
    df.drop(["PassengerId", "Name", "Ticket", "Cabin", "Parch", "SibSp", 'Embarked', 'Fare'], inplace=True,axis=1)
    df = scaling_values(df)
    return df

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
LABEL = 'Survived'
mean_age = df_raw['Age'].mean()
df = transform_data(df_raw, mean_age)
X = df
X = X.drop('Survived', axis=1)
y = df['Survived']

k_fold = KFold(
    n_splits=N_SPLITS,
    shuffle=True,
    random_state=RANDOM_STATE
)

scores = []

for train_index, test_index in k_fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    clf = RandomForestClassifier(n_estimators=N_ESTIMATORS, bootstrap=True, criterion='entropy',
                                min_samples_leaf=MIN_SAMPLES_LEAF,
                                min_samples_split=MIN_SAMPLE_SPLIT, random_state=RANDOM_STATE)

    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)

    acc_score = round(accuracy_score(y_test, y_predict),3)

    print(acc_score)

    scores.append(acc_score)

print()
print("Average:", round(100*np.mean(scores), 1), "%")
print("Std:", round(100*np.std(scores), 1), "%")

0.793
0.831
0.865
0.792
0.831

Average: 82.2 %
Std: 2.7 %


## Accuracy felt down a bit but std is better kaggle score also improved from 0.76415 to 0.77033

## Model Selection

In [15]:
df = transform_data(df_raw, 30)

In [16]:
df

,Survived,Pclass,Age,RelativesOnboard,Title,is_male
0,0,0.827377,-0.592481,0.059160,0.214278,0.737695
1,1,-1.566107,0.638789,0.059160,1.504288,-1.355574
2,1,0.827377,-0.284663,-0.560975,-1.075732,-1.355574
3,1,-1.566107,0.407926,0.059160,1.504288,-1.355574
4,0,0.827377,0.407926,-0.560975,0.214278,0.737695
...,...,...,...,...,...,...
886,0,-0.369365,-0.207709,-0.560975,2.794297,0.737695
887,1,-1.566107,-0.823344,-0.560975,-1.075732,-1.355574
888,0,0.827377,0.000000,1.299429,-1.075732,-1.355574
889,1,-1.566107,-0.284663,-0.560975,0.214278,0.737695


In [17]:
label_encoder = LabelEncoder()
df['Title'] = label_encoder.fit_transform(df['Title'])

In [18]:
df

,Survived,Pclass,Age,RelativesOnboard,Title,is_male
0,0,0.827377,-0.592481,0.059160,3,0.737695
1,1,-1.566107,0.638789,0.059160,4,-1.355574
2,1,0.827377,-0.284663,-0.560975,2,-1.355574
3,1,-1.566107,0.407926,0.059160,4,-1.355574
4,0,0.827377,0.407926,-0.560975,3,0.737695
...,...,...,...,...,...,...
886,0,-0.369365,-0.207709,-0.560975,5,0.737695
887,1,-1.566107,-0.823344,-0.560975,2,-1.355574
888,0,0.827377,0.000000,1.299429,2,-1.355574
889,1,-1.566107,-0.284663,-0.560975,3,0.737695


In [19]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [20]:
models = [
    SVC(),
    SGDClassifier(),
    KNeighborsClassifier(),
    GaussianProcessClassifier(),
    GaussianNB(),
    DecisionTreeClassifier(),
    AdaBoostClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    MLPClassifier()
]

In [21]:
y = df['Survived']
X = df.drop('Survived', axis=1)
for model in models:
    
    k_fold = KFold(
        n_splits=N_SPLITS,
        shuffle=True,
        random_state=RANDOM_STATE
    )

    scores = []
    for train_index, test_index in k_fold.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]

        clf = model

        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)

        acc_score = round(accuracy_score(y_test, y_predict),3)

        #print(acc_score)

        scores.append(acc_score)

    print(model)
    print("Average:", round(100*np.mean(scores), 1), "%")
    print("Std:", round(100*np.std(scores), 1), "%")

SVC()
Average: 82.9 %
Std: 2.4 %
SGDClassifier()
Average: 76.5 %
Std: 4.0 %
KNeighborsClassifier()
Average: 81.1 %
Std: 1.9 %
GaussianProcessClassifier()
Average: 82.0 %
Std: 2.6 %
GaussianNB()
Average: 81.2 %
Std: 2.7 %
DecisionTreeClassifier()
Average: 79.1 %
Std: 2.2 %
AdaBoostClassifier()
Average: 81.5 %
Std: 1.1 %
RandomForestClassifier()
Average: 81.5 %
Std: 1.9 %
GradientBoostingClassifier()
Average: 81.5 %
Std: 1.7 %


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()
Average: 81.6 %
Std: 2.3 %


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
